In [166]:
### Installation des packages 
import json
import os
import pandas as pd
import re
import requests
from lxml import *

from bs4 import BeautifulSoup as bs

## Constante 

In [167]:
### 
PATH_DATA = r"/Users/haroldtankpinou/Documents/real_estate"
PATH_DATA_RAW = rf"{PATH_DATA}/Raw"


In [168]:
if os.path.exists(PATH_DATA_RAW): 
    print("Le dossier existe déjà")
else : 
    print("Le dossier n'existe pas")

Le dossier existe déjà


## Télécharger les données d'une page 

In [707]:
#url_citya = r"https://www.citya.com/annonces/location?sort=b.dateMandat&direction=desc&page={page}"
url_citya = r"https://www.citya.com/annonces/location/appartement,maison?sort=b.dateMandat&direction=desc&page=2"

response = requests.get(url_citya)


In [708]:
html = response.content

In [709]:

### Parser avec le parseur lxml 
soup = bs(html, 'html')
print(soup.prettify())

<!DOCTYPE html>
<html lang="fr">
 <head>
  <meta content="width=device-width, initial-scale=1.0, shrink-to-fit=no" name="viewport"/>
  <title>
   Location appartement ou maison : Annonce Immobilière  page 2 | Citya
  </title>
  <link href="https://cdn.agencereference.com" rel="preconnect"/>
  <link href="https://media.sas-arche.com" rel="preconnect"/>
  <script>
   (function (w, d, s, l, i) {
                    w[l] = w[l] || [];
                    w[l].push({
                        'gtm.start':
                            new Date().getTime(), event: 'gtm.js'
                    });
                    var f = d.getElementsByTagName(s)[0],
                        j = d.createElement(s), dl = l != 'dataLayer' ? '&l=' + l : '';
                    j.async = true;
                    j.src =
                        'https://www.googletagmanager.com/gtm.js?id=' + i + dl;
                    f.parentNode.insertBefore(j, f);
                })(window, document, 'script', 'dataLayer', 'GT

In [717]:
### Récupération des variables test sur une seule page et un seul bloc 
descs = soup.find_all('div', {'class': 'infos'})
descs = descs[10]
ville = descs.find('p',class_ = 'ville').text.split()[0].strip()
post_box = descs.find('p',class_ = 'ville').text.split()[1].strip()
prix = descs.find('p',class_ = 'prix').find('strong').text.replace('\u202f','').replace('\xa0€','').split().replace(',','.')
type_piece = descs.find('h3').text.split()[0]
nb_piece = descs.find('h3').find('strong').text.split() 

text = descs.find('span', class_ = 'description-more').text
type_piece = descs.find('h3').text.split()[0]
nb_piece = descs.find('h3').find('strong').text.split() 

if type_piece == 'Appartement':
    nb_piece = nb_piece[0]
    reg_surface = r'(\d+?,.\d+)\s*m²'
    surface_text = descs.find('h3').find('strong').text 
    reg_surf_search = re.search(reg_surface,surface_text)
    surface = reg_surf_search.group(1).replace(',','.')
else:
    nb_piece = None
    surface_text = None

AttributeError: 'list' object has no attribute 'replace'

In [719]:

if type_piece == 'Appartement':
    nb_piece = nb_piece[0]
    reg_surface = r'(\d+?,.\d+)\s*m²'
    surface_text = descs.find('h3').find('strong').text 
    reg_surf_search = re.search(reg_surface,surface_text)
    surface = reg_surf_search.group(1).replace(',','.')
else:
    nb_piece = None
    surface_text = None

AttributeError: 'NoneType' object has no attribute 'group'

In [718]:
nb_piece #= descs.find('h3').find('strong').text.split() 
surface_text

' 3 pièces 65.66m²'

In [703]:
prix_text = descs.find('p',class_ = 'prix').find('strong').text.split()
if len(prix_text) > 2:
    prix_text = prix_text[0] + prix_text[1]
    prix = float(prix_text.replace(',','.'))
elif len(prix_text) <= 2:
    prix = float(prix_text[0].replace(',','.'))
else:
    prix = None

In [712]:
descs.find('p',class_ = 'prix').find('strong').text.replace('\u202f','').replace('\xa0€','').split()

['881']

In [705]:
descs.find('p',class_ = 'prix').find('strong').text.split('€')

['1\u202f232,80\xa0', '']

In [576]:
reg_surface = r'(\d+?[,\d+]\s*)mm²'
surface_text = descs.find('h3').find('strong').text 
reg_surf_search = re.search(reg_surface,surface_text)
reg_surf_search
#surface = reg_surf_search.group(1)#.replace(',','.')

In [585]:
import re

a = 'je fais 35,5 m²'
reg_surface = r'(\d+(?:,\d+)?)\s*m²'
reg_surf_search = re.search(reg_surface, a)

if reg_surf_search:
    surface = reg_surf_search.group(1)
    print("Surface:", surface)
else:
    print("Aucune correspondance trouvée.")


Surface: 35,5


In [575]:
 descs.find('h3').find('strong').text#.split() 

' 1 pièce 35m²'

In [492]:
descs.find('h3').find('strong')

<strong> 6 pièces</strong>

In [493]:
descs.find('h3').text.split()[3]

'pièces'

In [494]:
prix

1.0

In [495]:
nb_piece

'6'

In [496]:
surface_text

' 6 pièces'

In [512]:
def extract_dpe(text = text) : 
    
    reg_dpe = r"usage standard\s*:\s*(?:entre\s*)?(\d{1,3}(?:\s*\d{3})*(?:,\d{1,2})?)\s*et\s*(\d{1,3}(?:\s*\d{3})*(?:,\d{1,2})?)\s*euros"
    reg_dpe_2 = r"usage standard\s*:\s*(\d{1,3}(?:\s*\d{3})*(?:,\d{1,2})?)\s*euros"
    
    dpe_reg_1 = re.search(reg_dpe, text)
    dpe_reg_2 = re.search(reg_dpe_2, text)
    dpe_prix_min = None
    dpe_prix_max = None

    if dpe_reg_1:
        dpe_prix_min = float(dpe_reg_1.group(1).replace('\xa0', '').replace(',','.'))
        dpe_prix_max = float(dpe_reg_1.group(2).replace('\xa0', '').replace(',','.'))
    elif dpe_reg_2:
        dpe_prix_min = dpe_reg_2.group(1)
        dpe_prix_max = dpe_prix_min
    
    return dpe_prix_min,dpe_prix_max
    
    # Il n'y a qu'un seul montant dans ce cas


In [513]:
dpe_prix_min = float(extract_dpe(text)[0])
dpe_prix_max = float(extract_dpe(text)[1])

TypeError: float() argument must be a string or a number, not 'NoneType'

In [ ]:
### Création de la boucle 

In [262]:
descs = soup.find_all('div', {'class': 'infos'})
descs[2]

<div class="infos">
<a href="/annonces/location/appartement/blois-41000/GES02340003-245" title="Voir Appartement 1 pièce 35m²">
<h3>
                    Appartement
                    <strong> 1 pièce 35m²</strong>
</h3>
<p class="ville">Blois (41000)</p>
<p class="prix">
<strong>457 €</strong> / mois charges comprises
                                    </p>
</a>
<div style="position: relative">
<a href="/annonces/location/appartement/blois-41000/GES02340003-245" title="Voir Appartement 1 pièce 35m²">
<p class="description-start">A louer grand studio dans le centre historique de Blois comprenant une pièce principale spacieuse avec placard, grande cuisine séparée [...]
                            <span class="description-more">avec placard de rangement et réfrigérateur et une salle de bains avec wc et branchement machine à laver. 
 Loyer de 457,00 euros par mois charges comprises dont 35,00 euros par mois de provision pour charges (soumis à la régularisation annuelle). 
 Soit avec Ass

In [561]:

descs = soup.find_all('div', {'class': 'infos'})
data = []
for i,desc in enumerate(descs): 
    content = {
    'id':i,
     'description' : None,
     'ville' : None,
     'prix': None,
     'pieces': None,
     'surface' : None,
     'dpe_prix_min': None,
     'dpe_prix_max': None,
    }
    content['ville'] = desc.find('p',class_ = 'ville').text.split()[0]
    prix_text = desc.find('p',class_ = 'prix').find('strong').text.split()
    if len(prix_text) > 2:
        prix_text = prix_text[0] + prix_text[1]
        content['prix'] = float(prix_text[0].replace(',','.'))
    elif len(prix_text) <= 2:
         content['prix'] = float(prix_text[0].replace(',','.'))
    else:
        content['prix'] = None
    
    
    text = desc.find('span', class_ = 'description-more').text
    content['description'] = text
    nb_piece = desc.find('h3').find('strong').text.split() 
    
    
    
    if nb_piece and len(nb_piece) > 1:
        nb_piece = nb_piece[0]
        surface_text = float(desc.find('h3').text.split()[3].replace('m²','').replace(',','.'))
    else:
        nb_piece = None
        surface_text = None
        
    content['pieces'] = nb_piece
    content['surface'] = surface_text
    content['dpe_prix_min'] = extract_dpe(text)[0]
    content['dpe_prix_max'] = extract_dpe(text)[1]
    data.append(content)

ValueError: could not convert string to float: 'pièces'

In [562]:
descs = soup.find_all('div', {'class': 'infos'})
data = []

for i, desc in enumerate(descs):
    content = {
        'id': i,
        'ville': None
    }
    content['ville'] = desc.find('p', class_= 'ville').text.split()[0]
    
    data.append(content)


In [563]:
data

[{'id': 0, 'ville': 'Arras'},
 {'id': 1, 'ville': 'Belfort'},
 {'id': 2, 'ville': 'Blois'},
 {'id': 3, 'ville': 'Brech'},
 {'id': 4, 'ville': 'Brest'},
 {'id': 5, 'ville': 'Camblain-Châtelain'},
 {'id': 6, 'ville': 'Cambrai'},
 {'id': 7, 'ville': 'Chalon-sur-Saône'},
 {'id': 8, 'ville': 'Chalon-sur-Saône'},
 {'id': 9, 'ville': 'Châteauroux'},
 {'id': 10, 'ville': 'Châteauroux'},
 {'id': 11, 'ville': 'Châteauroux'},
 {'id': 12, 'ville': 'Grézieu-la-Varenne'},
 {'id': 13, 'ville': 'Hénin-Beaumont'},
 {'id': 14, 'ville': 'Laon'},
 {'id': 15, 'ville': 'Lille'},
 {'id': 16, 'ville': 'Mâcon'},
 {'id': 17, 'ville': 'Mâcon'}]